In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import xlwings as xw
import time

In [28]:
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('disable-notifications')

# Medlab
# driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_driver_path = "chromedriver.exe"
driver = webdriver.Chrome(executable_path=chrome_driver_path, options = chrome_options)
url = 'https://connections.medlabme.com/event/medlab-middle-east-2024/exhibitors/RXZlbnRWaWV3XzY5MTc5Ng==?filters=RmllbGREZWZpbml0aW9uXzIxMjAwMQ%253D%253D%3ARmllbGRWYWx1ZV8xNzMxNTIwNQ%253D%253D%2CRmllbGRWYWx1ZV8xMzY2ODQ4OQ%253D%253D%2CRmllbGRWYWx1ZV8xNzMxNTIxMg%253D%253D%2CRmllbGRWYWx1ZV8xMzY2ODQ5NA%253D%253D%2CRmllbGRWYWx1ZV8xMzY2ODQ5Nw%253D%253D%2CRmllbGRWYWx1ZV8xMzY2ODUwMw%253D%253D%2CRmllbGRWYWx1ZV8xNzMxNTIxMw%253D%253D%2CRmllbGRWYWx1ZV8xMzY2ODUyMQ%253D%253D'
driver.get(url)
driver.maximize_window()

In [29]:
# Shadow Root, falls Programm es nicht findet, selber klicken
# shadow_root = driver.find_element(By.CSS_SELECTOR, "#usercentrics-root").shadow_root
# sel = '#uc-center-container > div.sc-kdBSHD.gsTflx > div > div.sc-dtBdUo.fzvttm > div > button.sc-dcJsrY.eyWWqo'
# accept = WebDriverWait(shadow_root, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,sel))) 
# accept.click()
accept_xpath = '//*[@id="modal-container-1705325769754-0.07986755175320881"]/div/div/div/div[1]/button[2]/span/span/span'
accept = driver.find_element(By.XPATH, accept_xpath)
accept.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="modal-container-1705325769754-0.07986755175320881"]/div/div/div/div[1]/button[2]/span/span/span"}
  (Session info: chrome=120.0.6099.217)


In [30]:
# Define a function to scroll to the bottom of the page
def scroll_to_bottom():
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Get the initial page height
initial_height = driver.execute_script("return document.body.scrollHeight")
# Perform initial scroll
scroll_to_bottom()
# Wait for some time to let the content load
time.sleep(2)
# Keep scrolling until the page height stops increasing
while True:
    # Scroll to the bottom of the page
    scroll_to_bottom()
    # Wait for some time to let the content load
    time.sleep(2)
    # Get the updated page height
    new_height = driver.execute_script("return document.body.scrollHeight")
    # Check if the page height has stopped increasing
    if new_height == initial_height:
        break
    # Update the initial height for the next iteration
    initial_height = new_height

In [31]:
div_xpath = '//*[@id="__next"]/div[3]/div/div[3]/div[2]/div/div/div/div[2]'

# Find the div element
div_element = driver.find_element(By.XPATH, div_xpath)

# Find all the a elements within the div
a_elements = div_element.find_elements(By.XPATH, ".//a")

# Extract text and links
text_list = [a.text for a in a_elements]
link_list = [a.get_attribute("href") for a in a_elements]

In [32]:
# extract austeller und booth
name = []
booth = []
for i in range(len(text_list)):
    name.append(text_list[i].split('\n')[0])
    booth.append(text_list[i].split('\n')[-1])

In [16]:
# Excel ausfüllen
wb = xw.Book('2023 AACC Ausstellerliste_Vorlage.xlsx')
sheet = wb.sheets[0]

A_col_range = sheet.range('A3:A{}'.format(3+len(name)))
A_col_range.value = [[value] for value in name]

I_col_range = sheet.range('I3:I{}'.format(3+len(booth)))
I_col_range.value = [[value] for value in booth]

In [17]:
# add hyperlinks
for i in range(len(link_list)):
    row = 3+i    
    cell_row, cell_column = row, 1
    cell_address = (cell_row, cell_column)
    sheet.range(cell_address).add_hyperlink(link_list[i], text_to_display=name[i])
    sheet.range(cell_address).api.Font.Bold = True
    sheet.range(cell_address).font.color = (0, 0, 0)
    sheet.range(cell_address).api.Font.Size = 14
    sheet.range(cell_address).api.Font.Name = 'Verdana'

In [33]:
# Iterate over the links
austeller_text = []
for link in link_list:
    # Open a new tab and switch to it
    driver.execute_script("window.open('', '_blank');")
    driver.switch_to.window(driver.window_handles[1])
    # Navigate to the link
    driver.get(link)
    # Wait for some time to let the content load (you can adjust this time)
    time.sleep(2)
    try:
        # Extract text from the webpage
        webpage_text = driver.find_element_by_tag_name('body').text
        austeller_text.append(webpage_text)
    except NoSuchWindowException:
        # Handle the case where the window is already closed
        print(f"Window for {link} is already closed.")
    # Close the tab
    driver.close()
    # Switch back to the original tab
    driver.switch_to.window(driver.window_handles[0])

In [43]:
land = []
for i in range(len(austeller_text)):
    land.append(austeller_text[i].split('Country\n')[-1].split('\n')[0])

In [45]:
B_col_range = sheet.range('B3:B{}'.format(3+len(land)))
B_col_range.value = [[value] for value in land]